# Content Based Recommendation

In [1]:
import os

In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [4]:
DATA_DIR = os.path.abspath('./datasets/output/data_2.csv')
OUTPUT_DIR = os.path.join('./datasets/output')

## Loading Data

In [5]:
df = pd.read_csv(DATA_DIR, index_col=0)
df.head()

,class,text
id,,
182,Interview,"﻿विडम्बना, ओलीको सत्ता सवारी पनि सिंहदरबारमा फ..."
127,Interview,﻿नेपाली कांग्रेसको सभापतिमा रामचन्द्र पौडेलको ...
85,Interview,﻿\nदुर्भाग्य नै भन्नुपर्छ हाम्रो प्राथमिकता पे...
162,Interview,"﻿नयनराज पाण्डे, लेखक\nविमोचन हुनै लागेको तपाईं..."
13,Interview,"﻿- भवन भट्ट, उपाध्यक्ष, गैर आवासीय नेपाली संघ\..."


In [6]:
df.dropna(inplace=True) # drop rows with invalid values

In [7]:
y = df['class']
X = df['text']

In [8]:
X[0]

'\ufeffविडम्बना, ओलीको सत्ता सवारी पनि सिंहदरबारमा फोटो झुण्ड्याउने मार्गतिरै अघि बढिरहेको छ।\nकल्पना गर्नुुस्, कसैले तपाईंको कन्चटमा पेस्तोल राख्दै तत्काल ‘एमाले अध्यक्ष तथा प्रधानमन्त्री केपी शर्मा ओली, एनेकपा माओवादीका अध्यक्ष पुष्पकमल दाहाल र नेपाली कांग्रेसका सभापति शेरबहादुर देउवामध्ये उत्कृष्ट को हो?’ भनी सोध्यो। के तपाईं निमेषभरमै एकजनाको नाम भन्न सक्नुहोला? जवाफ सहज छैन।\nहामीले माने पनि नमाने पनि वर्तमान नेपाली राजनीतिका त्रिमूर्ति यिनै हुन्। यिनकै विवेक, बेवकूफी, बेहोशीले नै नेपालको दशा र दिशा निर्धारित गरिरहेको छ।\nअहिले एमाले अध्यक्ष ओली सत्ता राजनीतिको केन्द्रमा छन्। पार्टी प्रमुख र प्रधानमन्त्रीमा उनलाई प्राप्त यो पहिलो मौका संभवतः अन्तिम पनि हो, जसमा उनकै स्वीकारोक्ति पटकपटक सुनिएकै छ।\nपार्टी नेतृत्वमा चुनिंदा अरूहरू जस्तो आफूसँग प्रशस्त समय नभएका कारण छिटोछिटो गर्ने, निःसन्तान रहेकाले धन सञ्चित गर्नु नपर्ने जस्ता भनाइले उनले चर्चा कमाएका थिए। पार्टी अध्यक्षका रूपमा संविधानलेखनमा देखाएको जुझारु भूमिकाले पनि नागरिकले स्वाभाविक रूपमा उनीबाट केही अपेक्षा राखेका थिए।\nतर, 

## Processing data

In [10]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [11]:
nepali_stop_words = set(stopwords.words('nepali'))

tf = TfidfVectorizer(stop_words=nepali_stop_words,encoding='utf-8',decode_error='ignore')
tf = TfidfVectorizer(tokenizer= lambda x: x.split(" "),
                                  sublinear_tf=True, encoding='utf-8',
                                  decode_error='ignore',
                                  max_df=0.5,
                                  min_df=10,
                                  stop_words=nepali_stop_words)

tfidf_matrix = tf.fit_transform(df['text'])

cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

results = {}

E:\Anaconda\lib\site-packages\sklearn\feature_extraction\text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['कम', 'से'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
E:\Anaconda\lib\site-packages\sklearn\metrics\pairwise.py:54: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float


In [28]:
for idx, row in df.iterrows():
    idx = int(idx)
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
    similar_items = [(cosine_similarities[idx][i], df['index'][i]) for i in similar_indices]

    results[row['index']] = similar_items[1:]
    
print('Completed')
similar_indices = cosine_similarities[idx].argsort()[:-100:-1]

KeyError: 'index'